# Task Lifecycle

This notebook demonstrates the task lifecycle for sub-agent execution. Instead of calling a sub-agent directly and waiting for its result, we submit tasks to a broker that manages state, persistence, and observation.

In [ ]:
from agentic_patterns.core.tasks import TaskBroker

## Submitting a Task

The broker is an async context manager. On entry it starts a background dispatch loop that picks up pending tasks and hands them to a worker. We submit a task with an input string and optional metadata that configures the sub-agent.

In [ ]:
async with TaskBroker() as broker:
    task_id = await broker.submit(
        "What are the three laws of thermodynamics? One sentence each.",
        system_prompt="You are a physics teacher. Be concise.",
    )
    print(f"Submitted task: {task_id[:8]}")

    # Wait for the task to reach a terminal state
    task = await broker.wait(task_id)
    print(f"State: {task.state.value}")
    print(f"Result: {task.result}")

## Observation: Streaming Events

Instead of waiting for the final result, we can stream events as they happen. The broker yields `TaskEvent` objects that record state transitions and progress.

In [ ]:
async with TaskBroker() as broker:
    task_id = await broker.submit(
        "Explain the difference between a stack and a queue.",
        system_prompt="You are a computer science instructor.",
    )

    async for event in broker.stream(task_id):
        print(f"[{event.event_type.value}] {event.payload}")

## Cancellation

Tasks can be cancelled before they complete. Cancelling a task transitions it to the `CANCELLED` terminal state.

In [ ]:
async with TaskBroker() as broker:
    task_id = await broker.submit(
        "Write a 2000-word essay on the history of computing.",
        system_prompt="You are a technology historian.",
    )

    task = await broker.cancel(task_id)
    print(f"State: {task.state.value}")